# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

In [1]:
!pip install tensorflow

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Latif-Calderón\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\tensorflow\\include\\external\\icu\\icu4c\\source\\common\\unicode\\appendable.h'
Consider using the `--user` option or check the permissions.



  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl (390.0 MB)


In [2]:
!pip install keras

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [16]:
# your code here
import pandas as pd

# 1.Read the dataset into a DataFrame
tic_tac_toe_data = pd.read_csv('tic-tac-toe.csv')

# 2. Display the first few rows of the dataset to determine reliability
print(tic_tac_toe_data.head())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


In [17]:
# Convert categorical values to numeric using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
tic_tac_toe_data = tic_tac_toe_data.apply(lambda col: label_encoder.fit_transform(col))
# Display the transformed dataset
print(tic_tac_toe_data.head())

   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   2   2   2   2   1   1   2   1   1      1
1   2   2   2   2   1   1   1   2   1      1
2   2   2   2   2   1   1   1   1   2      1
3   2   2   2   2   1   1   1   0   0      1
4   2   2   2   2   1   1   0   1   0      1


In [18]:
# 4. # Separate the inputs (X) and output (y) based on the provided column names
input_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
output_column = ['class']

X = tic_tac_toe_data[input_columns]
#tic_tac_toe_data ['class']
y = tic_tac_toe_data[output_column]

# View the separated inputs (X) and output (y)
print("Inputs (X):")
print(X.head())

print("\nOutput (y):")
print(y.head())

Inputs (X):
   TL  TM  TR  ML  MM  MR  BL  BM  BR
0   2   2   2   2   1   1   2   1   1
1   2   2   2   2   1   1   1   2   1
2   2   2   2   2   1   1   1   1   2
3   2   2   2   2   1   1   1   0   0
4   2   2   2   2   1   1   0   1   0

Output (y):
   class
0      1
1      1
2      1
3      1
4      1


In [19]:
# 5. Normalize the input data using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
# Print the first few rows of the normalized data
print(X_normalized[:5])  # Assuming you want to view the first 5 rows

[[ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
   1.00322257 -0.16731812 -0.28682739]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -0.28682739  1.08495342 -0.28682739]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -0.28682739 -0.16731812  1.00322257]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -0.28682739 -1.41958965 -1.57687736]
 [ 1.00322257  1.08495342  1.00322257  1.08495342 -0.42007679 -0.16731812
  -1.57687736 -0.16731812 -1.57687736]]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.keras`.

In [20]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
##use the train_test_split function to split the normalized input data (X_normalized) and the output data (y) into training and testing sets. The test_size parameter specifies the proportion of the dataset to include in the testing set.

# Display the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (766, 9)
X_test shape: (192, 9)
y_train shape: (766, 1)
y_test shape: (192, 1)


In [21]:
#2
from tensorflow.keras.models import Sequential

# Create a Sequential model
model = Sequential()

In [22]:
# After adding layers to the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
#3. 
from tensorflow.keras.layers import Dense

# Add layers to the Sequential model
## channge hyperparameters to fintune the algorithm 64/100
model.add(Dense(units=64, activation='relu', input_shape=(100,)))  # Example input_shape assuming the number of input features
model.add(Dense(units=32, activation='relu'))
# Add the output layer with Softmax activation for multi-class classification
model.add(Dense(units=3, activation='softmax'))  # Example num_classes representing the number of output classes

c:\Users\Latif-Calderón\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 64)             │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,643 (33.76 KB)

 Trainable params: 8,643 (33.76 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
#4. # Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In this code, the compile method configures the learning process for the model by specifying the optimizer, loss function, and metrics. Here, 'adam' is used as the optimizer, 'sparse_categorical_crossentropy' as the loss function, and 'accuracy' as the metric for evaluating the model's performance.

By compiling the model in this way, you prepare it for training and specify the metrics that will be utilized for assessing its performance on the training and validation data.

In [ ]:
#5. Fit the training data.
# Example of defining the model with the first layer specifying the input shape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_shape = X_train.shape[1]  # Determining the input shape based on the number of features

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu'),
    Dense(16, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.1571 - loss: 2.5549 - val_accuracy: 0.6406 - val_loss: 1.7924
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6716 - loss: 1.5525 - val_accuracy: 0.6510 - val_loss: 1.0757
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6501 - loss: 0.9677 - val_accuracy: 0.6615 - val_loss: 0.7451
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6698 - loss: 0.6965 - val_accuracy: 0.6823 - val_loss: 0.6508
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7052 - loss: 0.5982 - val_accuracy: 0.6927 - val_loss: 0.6136
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7277 - loss: 0.5729 - val_accuracy: 0.7031 - val_loss: 0.5927
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7544 - loss: 0.5431 - val_accuracy: 0.7188 - val_loss: 0.5756
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7852 - loss: 0.5125 - val_accuracy: 0.7500 - val_loss

 fit method trains the model using the training data over a specified number of epochs. Additionally, the validation_data parameter allows you to monitor the model's performance on the validation set during training.

After fitting the model, the training history is typically stored in the history variable, which can be used to visualize the model's training and validation metrics over the epochs.

In [30]:
#6. Evaluate your neural network model with the test data.
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Test accuracy: {accuracy:.4f}')
print(f'Test loss: {loss:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7315 - loss: 0.5371 
Test accuracy: 0.7448
Test loss: 0.5312


By calling the evaluate method with the test data, you can obtain the test accuracy and loss, providing insight into the model's performance on unseen data.

In [33]:
# your code here
# Save the model as tic-tac-toe.model
model.save('tic-tac-toe.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [44]:
import pandas as pd
import numpy as np

# ... (previous code to create X_test and y_test)
from sklearn.model_selection import train_test_split
# Assuming X_normalized contains the input features and y contains the target variable
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Convert X_test and y_test to DataFrames if they are NumPy arrays
X_test_df = pd.DataFrame(X_test, columns=['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])  # Replace with the actual column names
y_test_df = pd.DataFrame(y_test, columns=['class'])  # Replace 'class' with the actual output column name

# Reset the indices
X_test_df.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

# Generate predictions using the loaded model
predictions = loaded_model.predict(X_test_df)
# 'tic-tac-toe.keras' is a variable, Python interprets as an undefined name

# Compare predictions with actual labels
for i in range(len(predictions)):
    print(f'Predicted: {np.argmax(predictions[i])}, Actual: {y_test_df.iloc[i]}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Predicted: 1, Actual: class    0
Name: 0, dtype: int64
Predicted: 1, Actual: class    1
Name: 1, dtype: int64
Predicted: 1, Actual: class    1
Name: 2, dtype: int64
Predicted: 1, Actual: class    0
Name: 3, dtype: int64
Predicted: 0, Actual: class    0
Name: 4, dtype: int64
Predicted: 1, Actual: class    1
Name: 5, dtype: int64
Predicted: 1, Actual: class    1
Name: 6, dtype: int64
Predicted: 1, Actual: class    1
Name: 7, dtype: int64
Predicted: 1, Actual: class    1
Name: 8, dtype: int64
Predicted: 1, Actual: class    0
Name: 9, dtype: int64
Predicted: 1, Actual: class    1
Name: 10, dtype: int64
Predicted: 1, Actual: class    1
Name: 11, dtype: int64
Predicted: 1, Actual: class    1
Name: 12, dtype: int64
Predicted: 0, Actual: class    0
Name: 13, dtype: int64
Predicted: 1, Actual: class    1
Name: 14, dtype: int64
Predicted: 1, Actual: class    1
Name: 15, dtype: int64
Predicted: 1, Actual: class    1
Name: 16, dtype: int64
Predicted: 1, Actual

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [50]:
# your code here
#add more layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a new Sequential model
improved_model = Sequential([
    Dense(100, activation='relu', input_shape=(input_shape,)),
    Dense(80, activation='relu'),  # Additional hidden layer
    Dense(50, activation='relu'),   # Additional hidden layer
    Dense(30, activation='softmax')
])

# Compile the model
improved_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\Latif-Calderón\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
#6. Evaluate your neural network model with the test data.
loss, accuracy = improved_model.evaluate(X_test, y_test)

print(f'Test accuracy: {accuracy:.4f}')
print(f'Test loss: {loss:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0096 - loss: 3.4706      
Test accuracy: 0.0104
Test loss: 3.4698


In [52]:
improved_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 100)            │         1,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 80)             │         8,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 50)             │         4,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 30)             │         1,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,660 (57.27 KB)

 Trainable params: 14,660 (57.27 KB)

 Non-trainable params: 0 (0.00 B)

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here